In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Save the compressed .tsv.gz file directly inside the EMR notebook environment.

The %%bash command allows you to run Bash commands in a notebook cell.

After downloading, upload it to raw-redfin-data-emr S3 bucket  so it becomes available for further processing

This bucket acts as the landing zone (raw zone) in your data lake architecture.The file is ready for decompression, reading into PySpark, and further transformation in the next step.

In [1]:
%%bash
# This is a Bash cell
wget -O - https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz | aws s3 cp - s3://raw-redfin-data-emr/city_market_tracker.tsv000.gz

--2025-07-10 01:19:22--  https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz
Resolving redfin-public-data.s3.us-west-2.amazonaws.com (redfin-public-data.s3.us-west-2.amazonaws.com)... 3.5.76.239, 3.5.80.178, 52.92.205.154, ...
Connecting to redfin-public-data.s3.us-west-2.amazonaws.com (redfin-public-data.s3.us-west-2.amazonaws.com)|3.5.76.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 936421049 (893M) [binary/octet-stream]
Saving to: ‘STDOUT’

     0K .......... .......... .......... .......... ..........  0%  780K 19m32s
    50K .......... .......... .......... .......... ..........  0%  200K 47m57s
   100K .......... .......... .......... .......... ..........  0%  279M 31m59s
   150K .......... .......... .......... .......... ..........  0%  301M 24m0s
   200K .......... .......... .......... .......... ..........  0%  299M 19m12s
   250K .......... .......... .......... .......... ..........  0% 

In [ ]:
spark = SparkSession.builder.appName("RedfinDataAnalysis").getOrCreate()

In [ ]:
redfin_data = spark.read.csv("s3://raw-redfin-data-emr/city_market_tracker.tsv000.gz", header=True, inferSchema=True, sep= "\t")

In [ ]:
redfin_data.show()

In [ ]:
redfin_data.printSchema()

In [ ]:
redfin_data.columns

In [ ]:
df_redfin = redfin_data.select(['period_end','period_duration', 'city', 'state', 'property_type',
    'median_sale_price', 'median_ppsf', 'homes_sold', 'inventory', 'months_of_supply', 'median_dom', 'sold_above_list', 'last_updated'])
df_redfin.show(3)

In [ ]:
#check total number of rows
print(f"Total number of rows: {df_redfin.count()}")

In [ ]:
from pyspark.sql.functions import isnull
# Count null values in each column
null_counts = [df_redfin.where(isnull(col_name)).count() for col_name in df_redfin.columns]
null_counts

In [ ]:
# Display the results
for i, col_name in enumerate(df_redfin.columns):
    print(f"{col_name}: {null_counts[i]} null values")

In [ ]:
# Check for missing values in the entire DataFrame
remaining_count = df_redfin.na.drop().count()

print(f"Number of missing rows: {df_redfin.count() - remaining_count}")

In [ ]:
print(f"Total number of remaining rows: {remaining_count}")

In [ ]:
#remove na and count total number of remaining rows
df_redfin = df_redfin.na.drop()
print(f"Total number of rows: {df_redfin.count()}")

In [ ]:
# Count null values in each column to confirm if we have removed all na
null_counts = [df_redfin.where(isnull(col_name)).count() for col_name in df_redfin.columns]
null_counts

In [ ]:
from pyspark.sql.functions import year, month

#Extract year from period_end and save in a new column "period_end_yr"
df_redfin = df_redfin.withColumn("period_end_yr", year(col("period_end")))

#Extract month from period_end and save in a new column "period_end_month"
df_redfin = df_redfin.withColumn("period_end_month", month(col("period_end")))

In [ ]:
# Drop period_end and last_updated columns
df_redfin = df_redfin.drop("period_end", "last_updated")

In [ ]:
df_redfin.show(3)

In [ ]:
from pyspark.sql.functions import when

#let's map the month number to their respective month name.

df_redfin = df_redfin.withColumn("period_end_month", 
                   when(col("period_end_month") == 1, "January")
                   .when(col("period_end_month") == 2, "February")
                   .when(col("period_end_month") == 3, "March")
                   .when(col("period_end_month") == 4, "April")
                   .when(col("period_end_month") == 5, "May")
                   .when(col("period_end_month") == 6, "June")
                   .when(col("period_end_month") == 7, "July")
                   .when(col("period_end_month") == 8, "August")
                   .when(col("period_end_month") == 9, "September")
                   .when(col("period_end_month") == 10, "October")
                   .when(col("period_end_month") == 11, "November")
                   .when(col("period_end_month") == 12, "December")
                   .otherwise("Unknown")
                 )

In [ ]:
df_redfin.show(3)

In [ ]:
#let us write the final dataframe into our s3 bucket as a parquet file.
s3_bucket = "s3://redfin-transform-zone-yml/redfin_data.parquet"
df_redfin.write.mode("overwrite").parquet(s3_bucket)